***Задание 2. SQL.***

    ЗАДАЧА 1.

    SELECT count(distinct st_id) 
      FROM ( 
    SELECT st_id, DATE_TRUNC('month', timest) AS month 
      FROM peas 
     WHERE correct = true 
     GROUP BY st_id, month 
    HAVING count(correct)>=20 
    ) as st_cnt; 

    ЗАДАЧА 2.

    WITH user_stud AS (
    SELECT s.st_id,
           s.test_grp,
    (SELECT COUNT(p.correct) > 10 
       FROM peas p 
      WHERE p.st_id = s.st_id AND p.correct = true) as active_st,
    (SELECT COUNT(p.correct) >= 2 
       FROM peas p 
      WHERE p.st_id = s.st_id AND p.correct = true AND p.subject = 'Math') as math_active_st,
    COALESCE((SELECT SUM(f.money) 
                FROM final_project_check f 
               WHERE f.st_id = s.st_id), 0) as revenue,
    (SELECT COUNT(*) > 0
       FROM final_project_check f
      WHERE f.st_id = s.st_id) as purchase,
    (SELECT COUNT(*) > 0
       FROM final_project_check f
      WHERE f.st_id = s.st_id AND f.subject = 'Math') as math_purchase
     FROM studs s
    )

    SELECT 
        test_grp,
        SUM(revenue) / COUNT(*) as ARPU,
        SUM(CASE WHEN active_st THEN revenue ELSE 0 END) / 
        SUM(CASE WHEN active_st THEN 1 ELSE 0 END) as ARPAU,
        SUM(purchase::int) * 100.0 / COUNT(*) as CR,
        SUM(CASE WHEN active_st THEN purchase::int ELSE 0 END) * 100.0 / 
        SUM(CASE WHEN active_st THEN 1 ELSE 0 END) as CR_active,
        SUM(CASE WHEN math_active_st THEN math_purchase::int ELSE 0 END) * 100.0 / 
        SUM(CASE WHEN math_active_st THEN 1 ELSE 0 END) as CR_math
      FROM user_stud
     GROUP BY test_grp
     ORDER BY test_grp;